# Tagowanie części mowy - Praca z tagami i biblioteką Numpy

W tym notebooku stworzymy macierz z wykorzystaniem niektórych informacji zawartych w tagach, a następnie zmodyfikujemy ją przy użyciu różnych podejść.
Będzie to służyć jako praktyczne doświadczenie w pracy z Numpy oraz jako wprowadzenie do niektórych elementów używanych do tagowania części mowy (POS - part of speech tagging).

In [1]:
import numpy as np
import pandas as pd

### Some information on tags

W przypadku tego notebooka będzie używany uproszczony przykład zawierający tylko trzy tagi (lub stany). W rzeczywistym zastosowaniu jest wiele innych tagów, które można znaleźć [tutaj](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html).

In [2]:
# Definiujemy oznaczenie tagów dla przysłówków RB, rzeczowników NN i przyimków TO
tags = ['RB', 'NN', 'TO']

W tym tygodniu skonstruujesz kilka słowników, które dostarczą użytecznych informacji o tagach i słowach, z którymi będziesz pracować. 

Jednym z tych słowników jest `transition_counts`, który liczy, ile razy dany tag zdarzył się obok innego. Klucze tego słownika mają postać `(previous_tag, tag)` a wartościami są częstotliwości występowania.

Inny jest słownik `emission_counts`, który będzie liczył ile razy dana para `(tag, słowo)` pojawiła się w zbiorze danych szkoleniowych.

Ogólnie rzecz biorąc, należy myśleć o `transition` tylko podczas pracy z tagami oraz o `emission` podczas pracy z tagami i słowami.

W tym notebooku będziesz patrzył na pierwszy z nich:

In [7]:
# Definiujemy słownik 'transition_counts' 
# Uwaga: wartości są takie same jak w zadaniu końcowym
transition_counts = {
    ('NN', 'NN'): 16241,
    ('RB', 'RB'): 2263,
    ('TO', 'TO'): 2,
    ('NN', 'TO'): 5256,
    ('RB', 'TO'): 855,
    ('TO', 'NN'): 734,
    ('NN', 'RB'): 2431,
    ('RB', 'NN'): 358,
    ('TO', 'RB'): 200
}

Zauważ, że istnieje 9 kombinacji 3 używanych tagów. Każdy tag może pojawić się po tym samym tag, więc należy je również uwzględnić.

### Użycie Numpy do stworzenia macierzy

Teraz stworzysz macierz, która zawiera te częstotliwości przy użyciu wektorów Numpy:

In [3]:
# zapisujemy liczbę wszystkich tagów w zmiennej 'num_tags'
num_tags = len(tags)

# Inicjalizujemy macierz 3x3 wypełnioną zerami
transition_matrix = np.zeros((num_tags, num_tags))

# Wypisujemy macierz
transition_matrix

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

Wizualnie można zobaczyć, że macierz ma odpowiednie wymiary. Nie zapomnij też, że możesz to sprawdzić używając  `shape`:

In [4]:
# Wymiary macierzy
transition_matrix.shape

(3, 3)

Przed wypełnieniem tej macierzy wartościami słownika `transition_counts` należy posortować tagi tak, aby ich umieszczenie w macierzy było spójne:

In [5]:
# Posortowana lista tagów 
sorted_tags = sorted(tags)

# Zobaczmy jak wygląda lista tagów po posortowaniu
sorted_tags

['NN', 'RB', 'TO']

Aby wypełnić tę matrycę właściwymi wartościami, możesz użyć `podwójnej pętli for`. Możesz również użyć `itertools.product` aby zapisać taką podwójną pętlę za pomocą jednej linijki kodu:

In [8]:
# pętla po wierszach
for i in range(num_tags):
    # pętla po kolumnach
    for j in range(num_tags):
        # Definiujemy parę tagów
        tag_tuple = (sorted_tags[i], sorted_tags[j])
        # Do pozycji (i,j) w macierzy 'transition_counts' zapisujemy częstotliwość ze słownika dla danej pary tagów
        transition_matrix[i, j] = transition_counts.get(tag_tuple)

# macierz po wypełnieniu danymi ze słownika
transition_matrix

array([[1.6241e+04, 2.4310e+03, 5.2560e+03],
       [3.5800e+02, 2.2630e+03, 8.5500e+02],
       [7.3400e+02, 2.0000e+02, 2.0000e+00]])

Wygląda na to, że to zadziałało dobrze. Jednak macierz może być trudna do odczytania, ponieważ `Numpy` skupia się raczej na wydajności, a nie na prezentowaniu wartości w ładnym formacie. 

Do tego celu można użyć `Pandas DataFrame`. W szczególności, funkcja, która bierze macierz jako wejście i wypisuje jej ładną wersję, będzie bardzo użyteczna:

In [9]:
# tworzymy funkcję 'print_matrix' do ładnego formatowania macierzy
def print_matrix(matrix):
    print(pd.DataFrame(matrix, index=sorted_tags, columns=sorted_tags))

Zauważ, że tagi nie są parametrem funkcji. Dzieje się tak, ponieważ lista `sorted_tags` nie zmieni się w pozostałej części notebooka, więc bezpieczne jest używanie wcześniej zadeklarowanej zmiennej. Aby przetestować tę funkcję, po prostu uruchom ją: 

In [10]:
# Sprawdźmy jak zadziała funkcja 'print_matrix'
print_matrix(transition_matrix)

         NN      RB      TO
NN  16241.0  2431.0  5256.0
RB    358.0  2263.0   855.0
TO    734.0   200.0     2.0


To wygląda o wiele lepiej, prawda? 

Jak już może zauważyłeś, ta matryca nie jest symetryczna.

### Praca na macierzach z Numpy

Teraz, gdy masz już ustawioną macierz, czas zobaczyć jak można nią manipulować po jej utworzeniu. 

`Numpy` pozwala na operacje wektorowe, co oznacza, że operacje, które normalnie obejmowałyby pętlę po macierzy, mogą być wykonywane w prostszy sposób. Jest to zgodne z traktowaniem macierzy Numpy jako macierzy, ponieważ otrzymuje się wsparcie dla wspólnych operacji na macierzach. Możesz robić mnożenie macierzy, mnożenie skalarne, dodawanie wektorów i wiele innych!

Na przykład spróbuj przeskalować każdą wartość w macierzy o współczynnik $\frac{1}{10}$. Normalnie zapętlałbyś każdą z wartości w macierzy, aktualizując je odpowiednio. Ale w Numpy jest to tak proste jak podzielenie całej macierzy przez 10:

In [11]:
# Skalowanie macierzy
transition_matrix = transition_matrix/10

# macierz po skalowaniu
print_matrix(transition_matrix)

        NN     RB     TO
NN  1624.1  243.1  525.6
RB    35.8  226.3   85.5
TO    73.4   20.0    0.2


Innym trudniejszym przykładem jest normalizacja każdego wiersza tak, aby każda wartość była równa $\frac{value}{sum \,of \,row}$.

Można to łatwo zrobić z wektoryzacją. Najpierw obliczysz sumę każdego wiersza:

In [12]:
# dla każdego rzędu obliczamy sumę wartości w rzędu
rows_sum = transition_matrix.sum(axis=1, keepdims=True)

# wypiszmy obliczone sumy
rows_sum

array([[2392.8],
       [ 347.6],
       [  93.6]])

Zauważ, że została użyta metoda `sum()`. Metoda ta robi dokładnie to, co sugeruje jej nazwa. Ponieważ suma wierszy była pożądana, oś została ustawiona na `1`. W Numpy `axis=1` odnosi się do kolumn, więc suma jest robiona przez sumowanie każdej kolumny danego wiersza, dla każdego wiersza. 

Również parametr `keepdims` został ustawiony na `True`, więc wynikowa macierz miała kształt `(3, 1)` a nie `(3,)`. Zostało to zrobione tak, aby osie były zgodne z pożądaną operacją. 

Podczas pracy z Numpy, zawsze pamiętaj o sprawdzeniu kształtu tablic, z którymi pracujesz, wiele nieoczekiwanych błędów zdarza się z powodu braku spójności osi. Atrybut `shape` będzie twoim przyjacielem dla tych przypadków.

In [13]:
# normalizacja transition matrix
transition_matrix = transition_matrix / rows_sum

# macierz po normalizacji
print_matrix(transition_matrix)

          NN        RB        TO
NN  0.678745  0.101596  0.219659
RB  0.102992  0.651036  0.245972
TO  0.784188  0.213675  0.002137


Zauważ, że przeprowadzona normalizacja wymusza, aby suma każdego rzędu była równa `1`. Można to łatwo sprawdzić, uruchamiając metodę "sum" na wynikowej macierzy:

In [14]:
transition_matrix.sum(axis=1, keepdims=True)

array([[1.],
       [1.],
       [1.]])

Dla ostatecznego przykładu, jesteś proszony o modyfikację każdej wartości przekątnej macierzy tak, aby były one równe `log` sumy bieżącego wiersza plus bieżąca wartość. Wykonując takie operacje matematyczne jak ta, nie zapomnij zaimportować modułu `math`. 

Można to zrobić przy użyciu standardu `pętli` lub`wektoryzacji`. Zobacz oba podejścia:

In [16]:
import math

# kopiujemy macierz, aby pokazać podejście z pętlą for
t_matrix_for = np.copy(transition_matrix)

# kopiujemy macierz, aby pokazać podejście z wykorzystaniem wektorów numpy
t_matrix_np = np.copy(transition_matrix)

#### Pętla for

In [17]:
# przechodzimy pętlą po wszystkich wartościach na przekątnej macierzy
for i in range(num_tags):
    t_matrix_for[i, i] =  t_matrix_for[i, i] + math.log(rows_sum[i])

# macierz po działaniu
print_matrix(t_matrix_for)

          NN        RB        TO
NN  8.458964  0.101596  0.219659
RB  0.102992  6.502088  0.245972
TO  0.784188  0.213675  4.541167


#### Działania na wektorach numpy

In [18]:
# Zapisujemy przekątną jako wektor numpy
d = np.diag(t_matrix_np)

# wypiszmy rozmiar przekątnej d
d.shape

(3,)

Możesz zapisać przekątną w tablicy numpy używając funkcji Numpy's `diag()`. Zauważ, że tablica ta ma kształt `(3,)`, więc jest niezgodna z wymiarami tablicy `rows_sum`, które są `(3, 1)`. Będziesz musiał zmienić kształt, zanim przejdziesz do przodu. Do tego celu można użyć funkcji Numpy `reshape()`, określającej pożądany kształt w tuple:

In [19]:
# zmieniamy rozmiar przekątnej d na (3,1)
d = np.reshape(d, (3,1))

# rozmiar d po zmianie
d.shape

(3, 1)

Teraz, gdy przekątna ma prawidłowy rozmiar, możesz wykonać operację wektorową poprzez zastosowanie funkcji `math.log()` do tablicy `rows_sum` i dodanie przekątnej. 

Aby zastosować funkcję do każdego elementu tablicy numpy, użyj funkcji Numpy `vektorize()` podającej żądaną funkcję jako parametr. Funkcja ta zwraca funkcję wektorową, która przyjmuje tablicę numpy jako parametr. 

Do aktualizacji oryginalnej macierzy można użyć funkcji Numpy `fill_diagonal()`.

In [20]:
# operacja na wektorach
d = d + np.vectorize(math.log)(rows_sum)

# Używamy funkcji 'fill_diagonal' aby wstawić dane na przekątnej
np.fill_diagonal(t_matrix_np, d)

# macierz po działaniu
print_matrix(t_matrix_np)

          NN        RB        TO
NN  8.458964  0.101596  0.219659
RB  0.102992  6.502088  0.245972
TO  0.784188  0.213675  4.541167


Aby sprawdzić, czy obie metody dają ten sam wynik, można porównać obie macierze. Zauważ, że ta operacja jest również wektorowa, więc uzyskasz sprawdzenie równości dla każdego elementu w obu matrycach:

In [21]:
# sprawdzenie równości
t_matrix_for == t_matrix_np

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

**Gratulujemy ukończenia tego notebooka!** Teraz powinieneś być bardziej zaznajomiony z niektórymi elementami używanymi przez model oznaczający części mowy, takimi jak słownik `transition_counts` oraz z pracą z Numpy.

**Kontynuuj dobrą robotę!**